In [1]:
import torch
import torch.nn as nn

import clip
import numpy as np


### 1. CLIP

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [9]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [22]:
model, preprocess = clip.load("ViT-B/32")
input_resolution = model.visual.input_resolution

print("Input resolution:", input_resolution)

Input resolution: 224


In [ ]:
trans = model.transformer
trans

### 2. Data

In [2]:
import math

In [4]:
x = torch.rand(20,10)
torch.max(x, dim=0, keepdim=True)[0]

tensor([[0.8934, 0.9792, 0.9816, 0.9600, 0.9860, 0.9865, 0.9661, 0.9618, 0.9940,
         0.9989]])

#### 2.1 CSI Dataset 

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_csi_data
from dataset.datasets import CSI_Dataset, HAR_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [2]:
# Widar
data_path = '../data'
batch_size = 20
time_length = 2000
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori', 'rec_dev']

In [ ]:
train_datas, train_labels, eval_datas, eval_labels = get_csi_data(
    data_path, 
    req_domains=req_domains,
)

train_labels.shape

In [4]:
# HAR
train_datas = HAR_Dataset('D:/NTU-Fi_HAR/train_amp', time_length=2000)

#### 2.2 DFS Data

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List
import random
import pywt

from dataset.data import get_dfs_data
from dataset.datasets import DFS_Dataset
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

In [2]:
dfs_data_path = '../dfs_data'
req_domains = ['users', 'gesture_type', 'tosor_loc', 'face_ori']

In [3]:
train_datas, train_labels, eval_datas, eval_labels = get_dfs_data(dfs_data_path, req_domains)

In [4]:
train_dataset = DFS_Dataset(
    dfs_data_path,
    train_datas,
    train_labels
)

### 2.3 BVP Data

In [1]:
import scipy.io as sio
import os
import numpy as np
from typing import List

from dataset.data import get_bvp_data

from dataset.datasets import CSI_Dataset, HAR_Dataset, BVP_Dataset
from torch.utils.data import DataLoader

In [2]:
train_datas, train_labels, eval_datas, eval_labels = get_bvp_data(
    '../bvp_data', 
    req_domains=['users', 'gesture_type', 'tosor_loc', 'face_ori']
)

In [3]:
data = BVP_Dataset('../bvp_data', train_datas, train_labels)
train_loader = DataLoader(data, batch_size=8, shuffle=True)

### 3.Model

In [1]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from model.embed import TokenEmbedding
from model.GPT2_Base import GPT2FCLS, build_GPT2FCLS

from einops import rearrange
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


d:\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### GPT2

In [ ]:
# GPT2 Model
model = GPT2Model.from_pretrained('gpt2')

### Qwen2.5-3B

In [ ]:
import torch
from transformers import Qwen2ForCausalLM, Qwen2Model

d:\Anaconda\envs\Deep_Learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model=Qwen2ForCausalLM.from_pretrained('unsloth/Qwen2.5-3B')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 2/2 [00:25<00:00, 12.99s/it]


### TimeNet

In [2]:
from model.embed import PatchEmbedding

In [30]:
xf = torch.fft.fft(x, dim=1) # x [batch, time, channel]
frequency_list = abs(xf).mean(0).mean(-1)
_, top_list = torch.topk(frequency_list, 3)
top_list = top_list.detach().cpu().numpy()
period = x.shape[1] // top_list

array([ 2,  3, 16], dtype=int64)